In [39]:
import warnings
import numpy as np
import scipy as sp
import pandas as pd
from scipy import stats
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve, ShuffleSplit
init_notebook_mode(connected=True)

pd.set_option("display.max_colwidth", 1000, "display.max_rows", None, "display.max_columns", None)
warnings.filterwarnings('ignore')

# 什么是机器学习

维基百科的定义：Machine learning is a field of artificial intelligence that uses **statistical** techniques to give computer systems the ability to **learn from data**, **without being explicitly programmed**.

# 为什么需要机器学习

- 个性化
- 知识挖掘
- 单调任务自动化
- ...

# 机器学习工作流

1. 指标定义（功能改进、新功能、新产品）
2. 数据获取
3. 数据准备（清洗、标注）
4. 模型构建
5. 模型评估
6. 模型部署

_迭代的过程_

## 数据获取

### 需要什么样的数据？
  - 已有数据
  - 开源数据
  - 爬虫
  - 第三方合作

### 需要多少数据？

#### 大数定理

In [79]:
np.random.seed(1)
p = 0.7 # 真实的正面向上的概率
trial_count = 1000
sizes = np.arange(1, trial_count)
rand_v = stats.bernoulli(p)
samples = [rand_v.rvs(size) for size in sizes]
estimated_p = [np.sum(sample)/len(sample) for sample in samples]

In [92]:
trace_0 = go.Scatter(x = sizes, y = estimated_p, mode = "lines", name = "估计的p值")
trace_1 = go.Scatter(x = sizes, y = np.full(trial_count, p), mode = "lines", name = f"真实的p值：{p}")
data = [trace_0, trace_1]
layout = go.Layout(
    xaxis = dict(title = "投掷次数（样本数）"),
    yaxis = dict(title = "正面向上的频率")
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

#### 学习曲线

In [93]:
def plot_learning_curve(model, X, y, cv=None, n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    train_sizes, train_scores, test_scores = learning_curve(model, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    # 两个方差之内的区域
    p1 = go.Scatter(x=train_sizes, y=test_scores_mean+test_scores_std, mode="lines", 
                    line=dict(color="blue", width=1), showlegend=False)
    p2 = go.Scatter(x=train_sizes, y=test_scores_mean-test_scores_std, mode="lines",
                    line=dict(color="blue", width=1), showlegend=False, fill="tonexty")
    p3 = go.Scatter(x=train_sizes, y=train_scores_mean+train_scores_std, mode="lines",
                    line=dict(color="red", width=1), showlegend=False)
    p4 = go.Scatter(x=train_sizes, y=train_scores_mean-train_scores_std, mode="lines",
                    line=dict(color="red", width=1), showlegend=False, fill="tonexty")
    # 均值
    p5 = go.Scatter(x=train_sizes, y=test_scores_mean, marker=dict(color="blue"),
                    name="cross validation score", showlegend=True)
    p6 = go.Scatter(x=train_sizes, y=train_scores_mean, marker=dict(color="red"),
                    name="training score", showlegend=True)
    data = [p1, p2, p3, p4, p5, p6]
    layout = go.Layout(
        title = "learning curve",
        xaxis = dict(title = "training size"),
        yaxis = dict(title = "score")
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [98]:
digits = load_digits()
# print(digits["DESCR"])
X, y = digits.data, digits.target
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)
model = SVC(gamma=0.001)
plot_learning_curve(model, X, y, cv=cv, n_jobs=8)

#### 一些参考方法

- 参考相似问题的解决方案

  例如论文
  
  
- 根据特征数量、分类个数等估算

  每一个分类都应该有足量的独立样本
  
  每引入一个新的特征，需要更多百分比的样本
  
  
- 模型越复杂，需要的数据一般越多

  例如对应每个模型参数，应该有若干个独立的样本
  
  
- 寻求领域专家的建议

  哪些数据、多少数据才能囊括真实世界中变量间的关系

# 示例

## loan charge-off prediction

通过借款人及贷款信息预测其贷款是否销账(charge off vs fully paid)

### EDA

In [2]:
loans_df = pd.read_csv("accepted_2007_to_2018Q2.csv.gz", compression="gzip")

/Users/yinhang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning:

Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.



In [3]:
loans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2004091 entries, 0 to 2004090
Columns: 151 entries, id to settlement_term
dtypes: float64(113), object(38)
memory usage: 2.3+ GB


In [4]:
loans_df.sample(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
123395,20429486,NaN,17200.0,17200.0,17200.0,36 months,12.99,579.46,C,C1,emergency department RN,7 years,RENT,120000.0,Source Verified,Jul-2014,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=20429486,NaN,debt_consolidation,Debt consolidation,956xx,CA,8.25,2.0,Jul-1997,670.0,674.0,2.0,4.0,NaN,7.0,0.0,17945.0,74.5,19.0,w,0.00,0.00,19775.140000,19775.14,17200.00,2575.14,0.0,0.0,0.0,Dec-2015,10503.78,NaN,Mar-2017,564.0,560.0,0.0,4.0,1.0,Individual,NaN,NaN,NaN,0.0,50.0,35648.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24100.0,NaN,NaN,NaN,3.0,5941.0,3791.0,81.9,1.0,0.0,158.0,203.0,2.0,2.0,3.0,6.0,4.0,2.0,4.0,2.0,2.0,3.0,4.0,8.0,4.0,6.0,12.0,3.0,7.0,0.0,0.0,1.0,2.0,84.2,33.3,0.0,0.0,45399.0,35648.0,21000.0,21299.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1780667,75202691,NaN,10000.0,10000.0,10000.0,36 months,11.99,332.10,C,C1,Interior supervisor,9 years,MORTGAGE,55000.0,Not Verified,Mar-2016,Current,n,https://lendingclub.com/browse/loanDetail.action?loan_id=75202691,NaN,debt_consolidation,Debt consolidation,331xx,FL,17.15,0.0,Jun-1993,715.0,719.0,2.0,NaN,NaN,11.0,0.0,5701.0,30.0,16.0,w,4604.71,4604.71,6960.780000,6960.78,5395.29,1565.49,0.0,0.0,0.0,Dec-2017,332.10,Jan-2018,Dec-2017,689.0,685.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,254.0,154065.0,2.0,2.0,0.0,0.0,30.0,14150.0,59.0,3.0,7.0,1746.0,46.0,19000.0,2.0,3.0,2.0,9.0,15407.0,7790.0,27.9,0.0,0.0,36.0,273.0,3.0,3.0,2.0,3.0,NaN,1.0,NaN,0.0,4.0,7.0,4.0,4.0,2.0,7.0,11.0,7.0,11.0,0.0,0.

In [5]:
loans_df["loan_status"].value_counts(dropna=False)

Current                                                1108697
Fully Paid                                              673112
Charged Off                                             170822
Late (31-120 days)                                       27678
In Grace Period                                          13775
Late (16-30 days)                                         7157
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     72
NaN                                                         29
Name: loan_status, dtype: int64

In [10]:
# 只考虑Fully Paid和Charged Off
loans = loans_df[loans_df["loan_status"].isin(["Fully Paid", "Charged Off"])]

In [11]:
loans.shape

(843934, 151)

In [12]:
loans["loan_status"].value_counts(dropna=False)

Fully Paid     673112
Charged Off    170822
Name: loan_status, dtype: int64

In [13]:
# 不平衡
loans["loan_status"].value_counts(normalize=True, dropna=False)

Fully Paid     0.797588
Charged Off    0.202412
Name: loan_status, dtype: float64

In [14]:
missing_fractions = loans.isnull().mean().sort_values(ascending=False)

In [15]:
missing_fractions.head(10)

member_id                                     1.000000
next_pymnt_d                                  1.000000
orig_projected_additional_accrued_interest    0.999879
hardship_start_date                           0.999297
hardship_length                               0.999297
hardship_type                                 0.999297
hardship_reason                               0.999297
hardship_status                               0.999297
deferral_term                                 0.999297
hardship_amount                               0.999297
dtype: float64

In [16]:
drop_list = sorted(list(missing_fractions[missing_fractions > 0.3].index))
print(drop_list)

['all_util', 'annual_inc_joint', 'debt_settlement_flag_date', 'deferral_term', 'desc', 'dti_joint', 'hardship_amount', 'hardship_dpd', 'hardship_end_date', 'hardship_last_payment_amount', 'hardship_length', 'hardship_loan_status', 'hardship_payoff_balance_amount', 'hardship_reason', 'hardship_start_date', 'hardship_status', 'hardship_type', 'il_util', 'inq_fi', 'inq_last_12m', 'max_bal_bc', 'member_id', 'mths_since_last_delinq', 'mths_since_last_major_derog', 'mths_since_last_record', 'mths_since_rcnt_il', 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq', 'next_pymnt_d', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'open_rv_12m', 'open_rv_24m', 'orig_projected_additional_accrued_interest', 'payment_plan_start_date', 'revol_bal_joint', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_earliest_cr_line', 'sec_app_fico_range_high', 'sec_app_fico_range_low', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_mths_since_last_ma

In [17]:
len(drop_list)

58

In [18]:
loans.drop(labels=drop_list, axis=1, inplace=True)

/Users/yinhang/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [19]:
loans.shape

(843934, 93)

In [20]:
print(sorted(loans.columns))

['acc_now_delinq', 'acc_open_past_24mths', 'addr_state', 'annual_inc', 'application_type', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'collection_recovery_fee', 'collections_12_mths_ex_med', 'debt_settlement_flag', 'delinq_2yrs', 'delinq_amnt', 'disbursement_method', 'dti', 'earliest_cr_line', 'emp_length', 'emp_title', 'fico_range_high', 'fico_range_low', 'funded_amnt', 'funded_amnt_inv', 'grade', 'hardship_flag', 'home_ownership', 'id', 'initial_list_status', 'inq_last_6mths', 'installment', 'int_rate', 'issue_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'last_pymnt_amnt', 'last_pymnt_d', 'loan_amnt', 'loan_status', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sa

In [22]:
keep_list = ['addr_state', 'annual_inc', 'application_type', 'dti',
             'earliest_cr_line', 'emp_length', 'emp_title', 
             'fico_range_high', 'fico_range_low', 'grade', 'home_ownership'
             , 'id', 'initial_list_status', 'installment', 'int_rate', 
             'issue_d', 'loan_amnt', 'loan_status', 'mort_acc', 'open_acc',
             'pub_rec', 'pub_rec_bankruptcies', 'purpose', 'revol_bal', 
             'revol_util', 'sub_grade', 'term', 'title', 'total_acc', 
             'verification_status', 'zip_code']

In [24]:
drop_list = [col for col in loans.columns if col not in keep_list]
loans.drop(labels=drop_list, axis=1, inplace=True)

In [25]:
loans.shape

(843934, 31)

In [30]:
print(sorted(loans.columns))

['addr_state', 'annual_inc', 'application_type', 'dti', 'earliest_cr_line', 'emp_length', 'emp_title', 'fico_range_high', 'fico_range_low', 'grade', 'home_ownership', 'id', 'initial_list_status', 'installment', 'int_rate', 'issue_d', 'loan_amnt', 'loan_status', 'mort_acc', 'open_acc', 'pub_rec', 'pub_rec_bankruptcies', 'purpose', 'revol_bal', 'revol_util', 'sub_grade', 'term', 'title', 'total_acc', 'verification_status', 'zip_code']


#### addr_state
借款人所在的州

In [36]:
loans["addr_state"].sample(5)

866173     IL
855867     CA
571020     CA
928629     NM
1308555    LA
Name: addr_state, dtype: object

In [35]:
loans["addr_state"].describe()

count     843934
unique        51
top           CA
freq      128537
Name: addr_state, dtype: object

In [37]:
loans["zip_code"].describe()

count     843934
unique       927
top        945xx
freq        9880
Name: zip_code, dtype: object

In [38]:
loans.drop(labels="zip_code", axis=1, inplace=True)

/Users/yinhang/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [27]:
loans["id"].describe()

count       843934
unique      843934
top       14680062
freq             1
Name: id, dtype: int64